# Exploratory Text Analysis

## What kinds of text analysis are there?

* analyst knows the pattern
    * regular expressions
* analyst does not know the pattern
    * natural language processing
        * compares historical examples to judge novel cases
            * comparisons are statistical and approximate
            

In [ ]:
# [a-zA-Z]{3}[ ]?[0-9]{4} -> 3 letters of any case, followed by 1 space (optional), followed by 4 nums!
# ?      means 0 or 1 occurance
# +      means 1 or more occurance
# *      means 0 or more occurance
# ^      means start of the pattern
# $      means end of the pattern

### Examples of Analysis

When you know the pattern:

In [ ]:
pattern = '£ ?[0-9][0-9]?' # £ then SPACE-optional then digit then digit-optional

document = 'My eggs cost £ 3, bread cost £2, vodka cost £ 35'

In [ ]:
import re

In [ ]:
re.findall(pattern, document)

['£ 3', '£2', '£ 35']

If you dont:

* sentiment analysis
    * how positive/negative is this (new) review?
* topic analysis
    * what is this document about?

## What can I do if I know what pattern I want to find?

* finding ("extracting")
    * what matches the pattern?
* matching ("validating")
    * does the entire document match YES/NO?
* substitue ("replacing")
    * replace a part that matches a pattern with another...

## How do I validate text with pandas?

In [ ]:
import pandas as pd

In [ ]:
ti = pd.read_csv('https://raw.githubusercontent.com/a-forty-two/diamler_17_jan/main/titanic.csv')
ti.sample(2)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
353,0,3,male,25.0,1,0,17.80,S,Third,man,True,NaN,Southampton,no,False
605,0,3,male,36.0,1,0,15.55,S,Third,man,True,NaN,Southampton,no,False


In [ ]:
ti['ticket'] = "Ticket: " + ti['class'] + "; Price: $ " + ti['fare'].astype(str) + "; Port: " + ti['embark_town'] + ";"

In [ ]:
ti[['class', 'fare', 'embark_town', 'ticket']].head(3)

,class,fare,embark_town,ticket
0,Third,7.2500,Southampton,Ticket: Third; Price: $ 7.25; Port: Southampton;
1,First,71.2833,Cherbourg,Ticket: First; Price: $ 71.2833; Port: Cherbourg;
2,Third,7.9250,Southampton,Ticket: Third; Price: $ 7.925; Port: Southampton;


In [ ]:
pattern = '(First|Second)'

ti['class'].str.match(pattern)

0      False
1       True
2      False
3       True
4      False
       ...  
886     True
887     True
888    False
889     True
890    False
Name: class, Length: 891, dtype: bool

In [ ]:
ti.loc[ ti['class'].str.match(pattern)  , 'survived'].mean()

0.5575

In [ ]:
pattern = '(Third)'
ti.loc[ ti['class'].str.match(pattern)  , 'survived'].mean()

0.24236252545824846

## How do I extract data with pandas?

In [ ]:
ti[['class', 'fare', 'embark_town', 'ticket']].head(3)

,class,fare,embark_town,ticket
0,Third,7.2500,Southampton,Ticket: Third; Price: $ 7.25; Port: Southampton;
1,First,71.2833,Cherbourg,Ticket: First; Price: $ 71.2833; Port: Cherbourg;
2,Third,7.9250,Southampton,Ticket: Third; Price: $ 7.925; Port: Southampton;


In [ ]:
#just for demo, no real purpose
pattern = '([0-9.]+)'

ti['ticket'].str.extract(pattern).sample(4)

,0
848,33.0
869,11.1333
405,21.0
651,23.0


## How do I substitue text with pandas?

In [ ]:
ti['ticket'].str.replace('$', '€').sample(1)

430    Ticket: First; Price: € 26.55; Port: Southampton;
Name: ticket, dtype: object

## What are regular expressions?

Regular expressions are a language for describing patterns in text.

They are separate from python, but may be used within python program. (And elsewhere, eg., often in SQL).

They are notoriously difficult to read and write; and as a separate language, an additional tool to learn.

## What regular expression patterns can I use?

* literals
    * `a`, find me an `a`
    * `£`, find `£`
    * `!` means `!`
    * ... most symbols mean "find me"
* `.`
    * find any **single** symbol
* character classes -- find a **single** symbol
    * `[abc]` $\rightarrow$ **either** a, b, c
    * `[0-9]` $\rightarrow$ **either** 0, 1, 2, 3,...9
    * `[A-Z]` $\rightarrow$ **either** capital A, B, ... Z
    * inversions
        * `[^abc]` $\rightarrow$ **is not** `a` OR `b` OR `c`
        * `[^a-zA-Z0-9 ]`  $\rightarrow$ **is not** alphanumeric-ish
    
* alternatives -- find the character**s** given by...
    * `(May|June|July)`  $\rightarrow$ **the whole worlds** May OR June..
    

In [ ]:
ti['ticket'].str.extract('(Ticket: (First|Second))')

In [ ]:
ti['ticket'].str.extract('( [0-9][0-9])')

In [ ]:
ti['ticket'].sample(1)

In [ ]:
ti['ticket'].str.extract('(Ticket: [A-Z])').sample(2)

In [ ]:
ti['ticket'].str.extract('(T........)').sample(3)

In [ ]:
ti['ticket'].str.extract('(Price: [^0-9A-Za-z] ..)').sample(3)

In [ ]:
ti['ticket'].str.extract('(Port: (Cherbourg|Southampton))').sample(3)

* repetitions
    * optional `?`
        * an optional number: `[0-9]?`
    * one or more `+`
        * one or more spaces: ` +`  
    * optional, or more, `*`
        * ` [0-9][0-9]?.[0-9]*`
    

In [ ]:
ti['ticket'].str.extract('([0-9][0-9]?.[0-9]*)').sample(3)

In [ ]:
ti['ticket'].str.extract('(Ticket: [a-zA-Z]+)').sample(3)

In [ ]:
row = 0
match = 1 # second match

ti['ticket'].str.extractall('([a-zA-Z]+: [a-zA-Z]+)').loc[row, match]

In [ ]:
ti['ticket'].str.extract('([a-zA-Z]+tow?n)')

* EXTRA:
    * escaping
        * How do I say, literally, the `.` symbol?
        * `\.`
    

In [ ]:
ti['ticket'].str.extract('(\$ [0-9]+\.[0-9]+)').sample(2)

* positional matching
    * `^` means **at the beginning**
    * `$` means **at the end**

In [ ]:
ti['ticket'].str.extractall('([a-zA-Z]+: [a-zA-Z]+;$)').sample(1)

In [ ]:
ti['ticket'].str.extractall('(^[a-zA-Z]+: [a-zA-Z]+;)').sample(1)

,,0
,match,
2,0,Ticket: Third;


## Next Steps

* review a "Regex Cheat Sheet"
    * also, eg., https://en.wikipedia.org/wiki/Regular_expression#Examples

## Exercise (30 min)

* find all the words in the tickets
    * HINT: a word is a repeated letter followed by a space or a colon
    * HINT: `[ :]` means a space or a colon
* find all the USD prices
    * HINT: ``` \$ ``` and repeated numbers
    
* find all the high-price tickets
    * HINT: consider `\$`, tripple-digit number, `\.`

In [ ]:
#Solution

In [48]:
ti['ticket'].str.findall('([a-zA-Z]+)').sample(10)

792    [Ticket, Third, Price, Port, Southampton]
539      [Ticket, First, Price, Port, Cherbourg]
205    [Ticket, Third, Price, Port, Southampton]
690    [Ticket, First, Price, Port, Southampton]
653     [Ticket, Third, Price, Port, Queenstown]
246    [Ticket, Third, Price, Port, Southampton]
423    [Ticket, Third, Price, Port, Southampton]
143     [Ticket, Third, Price, Port, Queenstown]
613     [Ticket, Third, Price, Port, Queenstown]
890     [Ticket, Third, Price, Port, Queenstown]
Name: ticket, dtype: object

In [49]:
ti['ticket'].str.extract('(\$ [0-9]+\.[0-9]+)').sample(2)

,0
251,$ 10.4625
873,$ 9.0


In [50]:
row = 0
match = 1 # second match
ti['ticket'].str.extractall('(\$ [0-9][0-9][0-9]+\.[0-9]+)')#.loc[row, 0]

,,0
,match,
27,0,$ 263.0
31,0,$ 146.5208
88,0,$ 263.0
118,0,$ 247.5208
195,0,$ 146.5208
215,0,$ 113.275
258,0,$ 512.3292
268,0,$ 153.4625
269,0,$ 135.6333
